In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
torch.manual_seed(1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cpu = torch.device('cpu')
print('Running on device: {}'.format(device))

torch.cuda.empty_cache() 

Running on device: cuda:0


In [2]:
X = []
Y = []
for p in tqdm(np.arange(0, 2), total = 2):
    X_p = torch.load('data_images/1face_X_part' + str(p) + '.pt', map_location = cpu)
    Y_p = torch.load('data_images/1face_Y_part' + str(p) + '.pt', map_location = cpu)
    X = X + [torch.stack(X_p)]
    Y = Y + Y_p
    del X_p

100%|██████████| 2/2 [01:23<00:00, 40.13s/it]


In [3]:
[x.shape for x in X]

[torch.Size([1269, 10, 3, 160, 160]), torch.Size([1577, 10, 3, 160, 160])]

In [4]:
X = torch.cat(X)

In [5]:
X.shape

torch.Size([2846, 10, 3, 160, 160])

In [6]:
len(Y)

2846

In [7]:
X = X.to(device)

In [8]:
from torch.utils.data import TensorDataset, DataLoader
dataset = TensorDataset(X[0:2840], torch.from_numpy(np.array(Y[0:2840])))
train_data, val_data = torch.utils.data.random_split(dataset, [2500, 340])
train_batch_size = 5
val_batch_size = 1
train_loader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=val_batch_size)

In [9]:
from torch import nn
import torch
import timm
# class Identity(nn.Module):
#     def __init__(self):
#         super(Identity, self).__init__()
#     def forward(self, x):
#         return x

# model_ft = timm.create_model("mixnet_m", pretrained=True)
# model_ft.classifier = Identity()
# batch, seqlen, c, h, w =  X.shape
# X = X.reshape(batch*seqlen, c, h, w) # .shape
# out = model_ft(X)
# out.shape
# out = out.reshape(batch, seqlen, out.shape[1])
# out.shape

In [10]:
import torch.nn as nn

class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.5):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.mixnet = timm.create_model("mixnet_m", pretrained=True)
        for param in self.mixnet.parameters():
                param.requires_grad = True
            
        self.lstm = nn.LSTM(1000, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.75)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_dim, 32)
#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(32, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size, seqlen, c, h, w = x.size()
        x = x.reshape(batch_size*seqlen, c, h, w).float()
        x = self.mixnet(x)
        x = x.reshape(batch_size, seqlen, x.shape[1])
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
#         out = self.batchnorm(out)
        out = self.fc1(out)
        out = self.elu(out)
#         out = self.fc2(out)
#         out = self.elu(out)
#         out = self.fc3(out)
#         out = self.elu(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [11]:
input_size = 512
output_size = 1
hidden_dim = 64
n_layers = 5

model = DFDCNet(input_size, output_size, hidden_dim, n_layers)
model.to(device)
train_criterion = nn.BCELoss()
val_criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

DFDCNet(
  (mixnet): EfficientNet(
    (conv_stem): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=24, bias=False)
          (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv_pw): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (1): Sequential(
        (0): InvertedResidual(
          (conv_pw): MixedConv2d(
            (0): Conv2d(12, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): Conv2d(1

In [12]:
# scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=500, verbose=True)


In [13]:
epochs = 5
counter = 0
print_every = 100
clip = .5
valid_loss_min = .5
val_loss = torch.tensor(np.Inf)
model.train()
for i in range(epochs):
    h = model.init_hidden(train_batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = train_criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(val_batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = val_criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './model_1face_unfroze.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
    scheduler.step(val_loss.item())


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 1/5... Step: 100... Loss: 0.017045... Val Loss: 0.231242
Validation loss decreased (0.500000 --> 0.231242).  Saving model ...
Epoch: 1/5... Step: 200... Loss: 0.019796... Val Loss: 0.224364
Validation loss decreased (0.231242 --> 0.224364).  Saving model ...
Epoch: 1/5... Step: 300... Loss: 0.028950... Val Loss: 0.215826
Validation loss decreased (0.224364 --> 0.215826).  Saving model ...
Epoch: 1/5... Step: 400... Loss: 1.038810... Val Loss: 0.218984
Epoch: 1/5... Step: 500... Loss: 0.595244... Val Loss: 0.208554
Validation loss decreased (0.215826 --> 0.208554).  Saving model ...
Epoch: 2/5... Step: 600... Loss: 0.037663... Val Loss: 0.216932
Epoch: 2/5... Step: 700... Loss: 0.840956... Val Loss: 0.212968
Epoch: 2/5... Step: 800... Loss: 0.023466... Val Loss: 0.214764
Epoch: 2/5... Step: 900... Loss: 0.940821... Val Loss: 0.222872
Epoch: 2/5... Step: 1000... Loss: 0.610154... Val Loss: 0.201687
Validation loss decreased (0.208554 --> 0.201687).  Saving model ...
Epoch: 3/5... 

# Reference
* https://github.com/ronghanghu/pytorch-gve-lrcn/blob/master/models/pretrained_models.py